### Tensorboard Simple example

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

In [2]:
config = tf.ConfigProto(device_count = {'GPU': 0})

def init_weights(shape,name):
    return tf.Variable( tf.random_normal(shape,stddev=0.01),name=name)

In [3]:
def model(X,w_h,W_h2,W_o,p_keep_input, p_keep_hidden):
    with tf.name_scope("layer1"):
        X = tf.nn.dropout(X,p_keep_input)
        h = tf.nn.relu(tf.matmul(X,w_h))
    with tf.name_scope("layer2"):
        h = tf.nn.dropout(h,p_keep_hidden)
        h2 = tf.nn.relu(tf.matmul(h,w_h2))
    with tf.name_scope("layer3"):
        h2 = tf.nn.dropout(h2,p_keep_hidden)
        return tf.matmul(h2,w_o)



In [4]:
#step1 get MNIST data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
trX,trY,teX,teY = mnist.train.images, mnist.train.labels,mnist.test.images,mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
#Step 2, create placeholders for inputs
X = tf.placeholder("float", [None, 784], name="X")
Y = tf.placeholder("float", [None, 10], name="Y")

In [6]:
#Step 3 : initialize weights
w_h = init_weights([784,625],"w_h")
w_h2= init_weights([625,625],"w_h2")
w_o=init_weights([625,10],"w_o")

In [7]:
#Step 4: Add histogram summaries for weights
tf.summary.histogram("w_h_summ",w_h)
tf.summary.histogram("w_h2_summ",w_h2)
tf.summary.histogram("w_o_summ",w_o)

<tf.Tensor 'w_o_summ:0' shape=() dtype=string>

In [8]:
#Step 5: Add dropout placeholders
p_keep_input = tf.placeholder("float",name="p_keep_input")
p_keep_hidden = tf.placeholder("float",name="p_keep_hidden")

In [9]:
#Step 6: Create model
py_x = model(X,w_h,w_h2,w_o,p_keep_input,p_keep_hidden)

In [10]:
#Step 7: Create cost function
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x,Y))
    train_op = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost)
    tf.summary.scalar("cost",cost)
#Step 8: Measure accuracy
with tf.name_scope("accuracy"):
    correct_pred = tf.equal( tf.argmax(Y,1) , tf.argmax(py_x,1) )
    acc_op = tf.reduce_mean(tf.cast(correct_pred,"float"))
    tf.summary.scalar("accuracy", acc_op)

In [11]:
#Step 9: Create a session
with tf.Session(config=config) as sess:
    #Step 10 create a log writer
    writer = tf.summary.FileWriter("./logs/nn_logs",sess.graph)
    #merged = tf.summary.merge([w_h_summ,w_h2_summ,w_o_summ, cost_summ, acc_summary])
    merged = tf.summary.merge_all()
        
    # Step 11 you need to initialize all variables
    tf.initialize_all_variables().run()

    #Step 12 train the  model
    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],p_keep_input: 0.8, p_keep_hidden: 0.5})
        
        acc = sess.run(acc_op, feed_dict={X: teX, Y: teY,p_keep_input: 1.0, p_keep_hidden: 1.0})
        summary = sess.run(merged, feed_dict={X: teX, Y: teY,p_keep_input: 1.0, p_keep_hidden: 1.0})

        writer.add_summary(summary, i)  # Write summary
        print(i, acc)                   # Report the accuracy    

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0 0.9366
1 0.9567
2 0.9683
3 0.9731
4 0.9774
5 0.9766
6 0.9784
7 0.9815
8 0.9792
9 0.9801
10 0.9814
11 0.9811
12 0.9816
13 0.9818
14 0.98
15 0.9823
16 0.9815
17 0.9813
18 0.9816
19 0.9817
20 0.9818
21 0.9832
22 0.9829
23 0.9826
24 0.9817
25 0.9814
26 0.9825
27 0.9825
28 0.9834
29 0.9838
30 0.9824
31 0.9827
32 0.9831
33 0.9835
34 0.9825
35 0.9828
36 0.9837
37 0.9837
38 0.983
39 0.9849
40 0.9839
41 0.9835
42 0.9825
43 0.983
44 0.9844
45 0.9832
46 0.9836
47 0.9825
48 0.9842
49 0.9845
50 0.9825
51 0.9844
52 0.9828
53 0.9852
54 0.9842
55 0.9849
56 0.9833
57 0.9851
58 0.9846
59 0.9847
60 0.9841
61 0.9831
62 0.9829
63 0.9836
64 0.9832
65 0.9837
66 0.9845
67 0.9855
68 0.9845
69 0.9834
70 0.9851
71 0.9832
72 0.9838
73 0.9844
74 0.9831
75 0.9849
76 0.9845
77 0.9842
78 0.9823
79 0.9845
80 0.9846
81 0.9849
82 0.9847
83 0.9866
84 0.9836
85 0.9847
86 0.9858
87 0.9843
88 0.9836
89 0.9835
90 0.9847
91 0.985
92 0.985
93 0.9843
94

In [12]:
sess.close()